In [46]:
T,M,max_sup_diff,cannot_be_together,must_have = read_file()
sorted_m = sortOnMIS(M)
sorted_m
L,supp = init_pass(T,sorted_m)
F=[]
FCount=[[]]
FtailCount=[[]]
F = calculate_F1(L,sorted_m,supp,must_have,F,T)
F
sorted_m

[('1', 0.003),
 ('2', 0.007),
 ('3', 0.01),
 ('4', 0.013),
 ('5', 0.016),
 ('6', 0.02),
 ('7', 0.023),
 ('8', 0.026),
 ('9', 0.029),
 ('10', 0.033),
 ('11', 0.036),
 ('12', 0.039),
 ('13', 0.042),
 ('14', 0.046),
 ('15', 0.049),
 ('16', 0.052),
 ('17', 0.056),
 ('18', 0.059),
 ('19', 0.062),
 ('20', 0.065),
 ('21', 0.069),
 ('22', 0.072),
 ('23', 0.075),
 ('24', 0.078),
 ('25', 0.082),
 ('26', 0.085),
 ('27', 0.088),
 ('28', 0.091),
 ('29', 0.095),
 ('30', 0.098),
 ('31', 0.101),
 ('32', 0.104),
 ('33', 0.108),
 ('34', 0.111),
 ('35', 0.114),
 ('36', 0.118),
 ('37', 0.121),
 ('38', 0.124),
 ('39', 0.127),
 ('40', 0.131),
 ('41', 0.134),
 ('42', 0.137),
 ('43', 0.14),
 ('44', 0.144),
 ('45', 0.147),
 ('46', 0.15),
 ('47', 0.153),
 ('48', 0.157),
 ('49', 0.16)]

In [2]:
import re
def read_file():
    T = {}
    M = {}
    cannot_be_together = []
    must_have = []
    t = 1
    with open("data2.txt") as file:
        for transaction in file:
            transList = []
            transarr = re.findall(r'\{(.*?)\}',transaction)
            str1 = ''.join(transarr)
            T[int(t)] = [x.strip() for x in str1.split(',')]
            t = t + 1
        
    with open("para2-1.txt") as param:
        param_data = [line for line in param]
    for d in param_data:
        if "MIS" in d:
            match = re.match(r'^.*\((.*)\).*= (\d*\.\d*)', d)
            M[match.group(1)] = float(match.group(2))
        elif "SDC" in d:
            max_sup_diff = float(re.match(r'.*= (.*)', d).group(1))
        elif "must-have" in d:
            must_have = [x.strip() for x in d.split(':')[1].split('or')]
        elif "cannot_be_together" in d:
            cannot_be_together = re.findall(r'{.*?}',d)
            
    return T,M,max_sup_diff,cannot_be_together,must_have

In [4]:
##final
from operator import itemgetter
from collections import OrderedDict
def sortOnMIS(M):
    #sorted_m = sorted(M.items(), key=operator.itemgetter(1))
    sorted_m = sorted(M.items(), key=itemgetter(1))
    return sorted_m

MIS(10) = 0.43
MIS(20) = 0.30
MIS(30) = 0.30
MIS(40) = 0.40
MIS(50) = 0.40
MIS(60) = 0.30
MIS(70) = 0.20
MIS(80) = 0.20
MIS(90) = 0.20
MIS(100) = 0.10
MIS(120) = 0.20
MIS(140) = 0.15

In [3]:
#M = {"10":0.43,"20":0.30,"30":0.30,"40":0.40,"50":0.40,"60":0.30,"70":0.20,"80":0.20,"90":0.20,"100":0.10,"120":0.20,"140":0.15}
#M

{'10': 0.43,
 '100': 0.1,
 '120': 0.2,
 '140': 0.15,
 '20': 0.3,
 '30': 0.3,
 '40': 0.4,
 '50': 0.4,
 '60': 0.3,
 '70': 0.2,
 '80': 0.2,
 '90': 0.2}

In [4]:
#sorted_m = sortOnMIS(M)
#sorted_m


[('100', 0.1),
 ('140', 0.15),
 ('70', 0.2),
 ('80', 0.2),
 ('90', 0.2),
 ('120', 0.2),
 ('20', 0.3),
 ('30', 0.3),
 ('60', 0.3),
 ('40', 0.4),
 ('50', 0.4),
 ('10', 0.43)]

In [5]:
## accessing values from tuples
##result = [m[1] for m in sorted_m if m[0] == '100' ]
##result

[0.1]

{1: ['{20,', '30,', '80,', '70,', '50,', '90}'], 2: ['{20,', '10,', '80,', '70}'], 3: ['{10,', '20,', '80}'], 4: ['{20,', '30,', '80}'], 5: ['{20,', '80}'], 6: ['{20,', '30,', '80,', '70,', '50,', '90,', '100,', '120,', '140}']}

In [6]:
#T={1:{'20', '30', '80', '70', '50', '90'},2:{'20', '10', '80', '70'},3:{'10', '20', '80'},4:{'20', '30', '80'},5:{'20', '80'},6:{'20', '30', '80', '70', '50', '90', '100', '120', '140'}}

In [7]:
#T

{1: {'20', '30', '50', '70', '80', '90'},
 2: {'10', '20', '70', '80'},
 3: {'10', '20', '80'},
 4: {'20', '30', '80'},
 5: {'20', '80'},
 6: {'100', '120', '140', '20', '30', '50', '70', '80', '90'}}

In [25]:
##final
def init_pass(T,sorted_m):
    L = []
    supp = calculate_support(T,sorted_m)
    first_sorted_m = sorted_m[0]
    len_t = len(T)
    L.append(first_sorted_m[0])
    mis_i = first_sorted_m[1]
    for m in sorted_m:
        if first_sorted_m[0] not in m:
            if supp[m[0]] >= mis_i:
                L.append(m[0])
    return L,supp

In [26]:
##final
def calculate_support(T,sorted_m):
    supp={}
    len_t = len(T)
    for m in sorted_m:
        count=0
        for t in T.values():
            if m[0] in t:
                count +=1
        supp[m[0]] = count/len_t
    return supp

L,supp = init_pass(T,sorted_m)
F = []
#change this
must_have=['20','40','50']
# F = calculate_F1(L,sorted_m,supp,must_have)
print(supp)
print(F)

In [27]:
def calculate_F1(L,sorted_m,supp,must_have,F,T):
    F1=[]
    fNewCount=[]
    fNewTailCount=[]
    for l in L:
        mis_l = [m[1] for m in sorted_m if m[0] == l]
        if supp[l] >= mis_l[0]:
            if must_have:
                if l in must_have:
                    F1.append(l)
            else:
                F1.append(l)
                
    
    
    F.append(F1)
    return F

In [47]:
def MS_candidate_gen(f,supp,max_sup_diff,sorted_m):
    C = []
    for f1 in f:
        for i in range(f.index(f1)+1,len(f)):
            f2 = f[i]
            if f1[len(f1)-1] != f2[len(f2)-1]:
                if supp[f1[len(f1)-1]] - supp[f2[len(f2)-1]] <= max_sup_diff:
                    mis_f1 = [m[1] for m in sorted_m if m[0] == f1[len(f1)-1]]
                    mis_f2 = [m[1] for m in sorted_m if m[0] == f1[len(f2)-1]]
                    if mis_f1 < mis_f2:
                        f1.append(f2[len(f2)-1])
                        C.append(f1)
                        c = C[len(C)-1]
                        tempS = findsubsets(c,len(c)-1)
                        print(tempS)
                        
            
    return C    

In [48]:
def level2_candidate_gen(L,max_sup_diff,supp,sorted_m):
    C2 = []
    for l in L:
        mis_l = [m[1] for m in sorted_m if m[0] == l]
        if supp[l] >= mis_l[0]:
            count = L.index(l) + 1
            for i in range(count, len(L)):
                if (supp[L[i]] >= mis_l[0] and abs(supp[L[i]] - supp[l]) <= max_sup_diff) :
                    C2.append([l,L[i]])
            count = count + 1
    return C2

In [49]:
def apply_item_constraints(F, cannot_be_together, must_have,fCount,fTailCount):
    F1 = []
    fNewCount=[]
    fNewTailCount=[]
    
    for f in F:
        remove = False
        if must_have:
            if set(f).intersection(set(must_have)):
                for items in cannot_be_together:
                    itemsarr = re.findall(r'\{(.*?)\}',items)
                    itemstr = ''.join(itemsarr)
                    itemList = [x.strip() for x in itemstr.split(",")]
                    if set(itemList).issubset(set(f)):
                        remove = True
                        break
                if not remove :
                    i = F.index(f)
                    if fCount:
                        fNewCount.append(fCount[i])
                    if fTailCount:
                        fNewTailCount.append(fTailCount[i])
                    F1.append(f)
                    
        else:
            for items in cannot_be_together:
                    itemsarr = re.findall(r'\{(.*?)\}',items)
                    itemstr = ''.join(itemsarr)
                    itemList = [x.strip() for x in itemstr.split(",")]
                    if set(itemList).issubset(set(f)):
                        remove = True
                        break
            if not remove :
                i = F.index(f)
                if fCount:
                    fNewCount.append(fCount[i])
                if fTailCount:
                    fNewTailCount.append(fTailCount[i])
                F1.append(f)
                
            
                
    return F1,fNewCount,fNewTailCount

In [54]:
## sorted_m - stores mis in sorted order
## T - Dictionary of transactions
## supp - Dictionary for all supports of elements in m
## L - List of L
## F1 - List of Frequent 1
def ms_apriori():
    T,M,max_sup_diff,cannot_be_together,must_have = read_file()
    sorted_m = sortOnMIS(M)
    L,supp = init_pass(T,sorted_m)
    F=[]
    FCount=[[]]
    FtailCount=[[]]
    F = calculate_F1(L,sorted_m,supp,must_have,F,T)
    k = 1 
    C=[[]]
    while F[k-1]:
        print(F[k-1])
        print(FCount[k-1])
        print(FtailCount[k-1])
        if k == 1:
            C.append(level2_candidate_gen(L,max_sup_diff,supp,sorted_m))
        else:
            C.append(MS_candidate_gen(F[k-1],supp,max_sup_diff,sorted_m))
        
        cCount=[0]*len(C[k])
        fCount=[]
        tailCount = [0]*len(C[k])
        fTailCount=[]
        for t in T.values():
            c_count = 0 
            for c in C[k]:
                if set(c).issubset(t):
                    cCount[c_count]=cCount[c_count]+1
                if set(c[1:]).issubset(t):
                    tailCount[c_count] = tailCount[c_count]+1
                
                c_count = c_count+1
        c_count=0
        f=[]
        for c in C[k]:
            mis_c1 = [m[1] for m in sorted_m if m[0] == c[0]]
            if cCount[c_count]/len(T) >= mis_c1[0]:
                f.append(c)
                fCount.append(cCount[c_count])
                fTailCount.append(tailCount[c_count])
            c_count = c_count+1
            
        F.append(f)
        FCount.append(fCount)
        FtailCount.append(fTailCount)
        print(F[k])
        f,fCount,fTailCount = apply_item_constraints(F[k], cannot_be_together, must_have,FCount[k],FtailCount[k])
        
        print(f)
        F.pop()
        FCount.pop()
        FtailCount.pop()
        F.append(f)
        FCount.append(fCount)
        FtailCount.append(fTailCount)
        
        k =k+1
    
    
    display(F,FtailCount,FCount)
    

In [55]:
ms_apriori()

['1', '2', '3', '13']
[]
[]
[['1', '30'], ['2', '25']]
[['1', '30'], ['2', '25']]
[['1', '30'], ['2', '25']]
[1, 2]
[13, 13]
[]
[]


NameError: name 'w' is not defined

In [65]:
T,M,max_sup_diff,cannot_be_together,must_have = read_file()
sorted_m = sortOnMIS(M)
L,supp = init_pass(T,sorted_m)
f= [['100','20'],['30','60'],['20','60']]
MS_candidate_gen(f,supp,max_sup_diff)

[['100', '20'], ['30', '60'], ['20', '60']]
1.0
0.0
1.0
0.0
0.0
0.0


In [71]:
c=['20']
set(c).issubset(T[1])

True

In [26]:
import itertools
def findsubsets(S,m):
    return set(itertools.combinations(S, m))

In [27]:
S=['100','200','300']
m=2
print(findsubsets(S,m))

{('100', '200'), ('100', '300'), ('200', '300')}


In [53]:
def display(F,tail_list,count_list):
    out_file = open("output-file", 'w')
    for k in F:
        out_file.write('Frequent ' + str(k) + '-itemsets\n')
        for f in F[k]:
            if k == 1:
                out_file.write('\n    ' + str(count[f[0]]) + ' : {' + ','.join(set(f)) + '}')
            else:
                tail_count = 0
                for tc in tail_list[k]:
                    tail_count = tc[f]
                for c in count_list[k]:
                    count = c[f]
                out_file.write("\n    " + str(count) + " : " + '{' + ', '.join(f) + '}')
                out_file.write("\nTailcount = " + str(tail_count))
        out_file.write("\n\n    Total number of frequent "+ str(k) + "-itemsets = " + str(len(F[k])) + "\n\n\n")